# Analysis of RNA-seq data

This `.ipynb` file should be placed in a directory titled `Code/`. 

In addition, one step *above* the `Code/` directory should be directories titled `Data/` and `Results/` such that file writing implemented here runs smoothly. 

So it doesn't matter where you put this directory, as long as the structure looks something like:

    my_cool_project/Code/
        RNA_seq_analysis.ipynb
        sequencing_analysis_library.py

    my_cool_project/Data/

    my_cool_project/Results/
    
In addition, in either your global path (if you don't know what this is fret not) OR your `my_cool_project/Code/` directory you will need to have `sequencing_analysis_library.py`

# Common imports / necessary helpful functions

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
#Common imports
from Bio import SeqIO
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats
import pandas as pd

#My helper library for this analysis
import sequencing_analysis_library as SAL

/Users/adamhockenberry/.pyenv/versions/anaconda3-2.0.1/lib/python3.4/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Reading the genome ... choose wisely

In [4]:
genome_file = '/Users/adamhockenberry/Projects/Neisseria/Data/Genomes/fa1090.gb'
# genome_file = '/Users/adamhockenberry/Projects/Neisseria/Code/temp_new_fa1090.gb'
genome = list(SeqIO.parse(genome_file, 'genbank'))[0]
organism = 'Neisseria'

# genome_file = '/Users/adamhockenberry/Projects/Neisseria/Data/Genomes/na1000.gb'
# genome = list(SeqIO.parse(genome_file, 'genbank'))[0]
# organism = 'Caulobacter'

genome_features_dict = SAL.get_genome_features(genome)

# Read TSS predator

In [ ]:
infile = '/Users/adamhockenberry/Downloads/TSSpredator_v1-04/04_01_16_NORPKM_more_sensitive/MasterTable.tsv'
tss_df = pd.read_csv(infile, sep='\t')
tss_df

In [ ]:
control_tss_dict = {}
treatment_tss_dict = {}
for index in tss_df.index[:]:
    if tss_df.iloc[index]['Genome'] == 'Normal':
        if tss_df.iloc[index]['detected'] == 1:
            location = tss_df.iloc[index]['SuperPos']
            if tss_df.iloc[index]['SuperStrand'] == '+':
                strand = 'plus'
            elif tss_df.iloc[index]['SuperStrand'] == '-':
                strand = 'minus'
            else:
                print('Error in strand')
            control_tss_dict[index] = (strand, location, location)
    
    elif tss_df.iloc[index]['Genome'] == 'Oxidative':
        if tss_df.iloc[index]['detected'] == 1:
            location = tss_df.iloc[index]['SuperPos']
            if tss_df.iloc[index]['SuperStrand'] == '+':
                strand = 'plus'
            elif tss_df.iloc[index]['SuperStrand'] == '-':
                strand = 'minus'
            else:
                print('Error in strand')
            treatment_tss_dict[index] = (strand, location, location)
    else:
        print('Error in genome')
print(len(control_tss_dict.keys()), len(treatment_tss_dict.keys()))

# Visualize data for individual genes

In [ ]:
###################
fwd_wiggle = '../Data/Neisseria/SQ-(1,3,5)_20_100_normed_f.wig'
rev_wiggle = '../Data/Neisseria/SQ-(1,3,5)_20_100_normed_r.wig'
fwd_dicty_ctrl, rev_dicty_ctrl =\
        SAL.read_single_wiggle(fwd_wiggle, rev_wiggle)

# ###################    
# fwd_wiggle = '../Data/Neisseria/SQ-(2,4,6)_20_100_normed_f.wig'
# rev_wiggle = '../Data/Neisseria/SQ-(2,4,6)_20_100_normed_r.wig'
# fwd_dicty_treatment, rev_dicty_treatment =\
#         SAL.read_single_wiggle(fwd_wiggle, rev_wiggle)

# ###################    
# fwd_wiggle = '../Data/Neisseria/SQ-(7,9,11)_20_100_normed_f.wig'
# rev_wiggle = '../Data/Neisseria/SQ-(7,9,11)_20_100_normed_r.wig'
# fwd_dicty_ctrl_drna, rev_dicty_ctrl_drna =\
#         SAL.read_single_wiggle(fwd_wiggle, rev_wiggle)

# ###################    
# fwd_wiggle = '../Data/Neisseria/SQ-(10,12)_20_100_normed_f.wig'
# rev_wiggle = '../Data/Neisseria/SQ-(10,12)_20_100_normed_r.wig'
# fwd_dicty_treatment_drna, rev_dicty_treatment_drna =\
#         SAL.read_single_wiggle(fwd_wiggle, rev_wiggle)




In [ ]:
feature_name = 'NGO0108'
# NGO1767 (catalase) 
# NGO1055
# NGO0114
# NGO0108
bases_to_add = 100
window_beg, window_end = (int(genome_features_dict[feature_name].location.start), int(genome_features_dict[feature_name].location.end))
vert_lines=[]
window_beg = window_beg-bases_to_add
window_end = window_end+bases_to_add
defined_ylimits = False
save_file = True
SAL.plot_individual_segment(window_beg, window_end, [genome_features_dict[feature_name]],\
                            [('Control', fwd_dicty_ctrl_drna, rev_dicty_ctrl_drna), ('Treatment', fwd_dicty_treatment_drna, rev_dicty_treatment_drna)],\
                            defined_ylimits=defined_ylimits, vert_lines=vert_lines, feature_name=feature_name, save_file=save_file)

In [ ]:
bases_to_add = 100
vert_lines=[]
defined_ylimits = False
save_file = False


for feature_name in list(genome_features_dict)[:]:
    window_beg, window_end = (int(genome_features_dict[feature_name].location.start), int(genome_features_dict[feature_name].location.end))
    window_beg = window_beg-bases_to_add
    window_end = window_end+bases_to_add
    SAL.plot_individual_segment(window_beg, window_end, [genome_features_dict[feature_name]],\
                                [('Ctrl', fwd_dicty_ctrl, rev_dicty_ctrl), ('H202', fwd_dicty_treatment, rev_dicty_treatment)],\
                                defined_ylimits=defined_ylimits, vert_lines=vert_lines, feature_name=feature_name, save_file=save_file)

# What about un-annotated genes?
The above analysis relied entirely on the published, annotated genome sequences. An interesting part of RNA-seq analysis is that perhaps we can find novel unannotated transcripts or sRNAs.

To do so, we'll have to at this stage brute force it. I recommend scanning in window of ~10,000 bps

# Treatment Vs Control

In [ ]:
window_beg, window_end = (50000, 60000)
# defined_ylimits = (5000, 5000)
# defined_ylimits = (200, 200)
# defined_ylimits = (20, 20)
defined_ylimits = False

feature_list = SAL.get_features_in_window(genome_features_dict, window_beg, window_end)

# vert_lines=[('minus', 2037736, 2038032)]
vert_lines=[]


feature_name=False
save_file=False
SAL.plot_individual_segment(window_beg, window_end, feature_list,\
                                [('Control', fwd_dicty_ctrl, rev_dicty_ctrl), ('Treatment', fwd_dicty_treatment, rev_dicty_treatment)],\
                                defined_ylimits=defined_ylimits, vert_lines=vert_lines, feature_name=feature_name,\
                            save_file=save_file)

feature_list.sort(key=lambda x: x.location.start)
print('Features shown on plot:')
print('Plus strand:')
for feature in feature_list:
    if feature.strand == 1:
        print(feature.qualifiers['locus_tag'][0], feature.location.start, feature.location.end)
print('Minus strand:')
for feature in feature_list:
    if feature.strand == -1:
        print(feature.qualifiers['locus_tag'][0], feature.location.start, feature.location.end)

# Tex treatment, control and treatment

In [ ]:
window_beg, window_end = (50000, 60000)
# defined_ylimits = (5000, 5000)
# defined_ylimits = (2000, 2000)
# defined_ylimits = (200, 200)
defined_ylimits = False
logy = False

feature_list = SAL.get_features_in_window(genome_features_dict, window_beg, window_end)

vert_lines_control = []
for i in control_tss_dict.values():
    if i[1] > window_beg and i[1] < window_end:
        vert_lines_control.append(i)
vert_lines_treatment = []
for i in treatment_tss_dict.values():
    if i[1] > window_beg and i[1] < window_end:
        vert_lines_treatment.append(i)

feature_name='NoRPKM_more_sensitive_control'
save_file=True


SAL.plot_individual_segment(window_beg, window_end, feature_list,\
                                [('Control', fwd_dicty_ctrl, rev_dicty_ctrl), ('Tex-Control', fwd_dicty_ctrl_drna, rev_dicty_ctrl_drna)],\
                                defined_ylimits=defined_ylimits, logy=logy, vert_lines=vert_lines_control, feature_name=feature_name,\
                            save_file=save_file)

feature_name='NoRPKM_more_sensitive_h202'
SAL.plot_individual_segment(window_beg, window_end, feature_list,\
                                [('h2o2', fwd_dicty_treatment, rev_dicty_treatment), ('Tex-h202', fwd_dicty_treatment_drna, rev_dicty_treatment_drna)],\
                                defined_ylimits=defined_ylimits, logy=logy, vert_lines=vert_lines_treatment, feature_name=feature_name,\
                            save_file=save_file)

###To show the feature names
feature_list.sort(key=lambda x: x.location.start)
print('Features shown on plot:')
print('Plus strand:')
for feature in feature_list:
    if feature.strand == 1:
        print(feature.qualifiers['locus_tag'][0], feature.location.start, feature.location.end)
print('Minus strand:')
for feature in feature_list:
    if feature.strand == -1:
        print(feature.qualifiers['locus_tag'][0], feature.location.start, feature.location.end)

# Analysis of hypothetical sRNAs

In [ ]:
save_file = False
defined_ylimits=False

srna_file = open('../Data/candidate_srna_neisseria.txt', 'r')
counter = 0
for line in srna_file:
    counter += 1
    if counter > 20:
        break
    split_line = line.split('|')
    try:
        name = split_line[0].split()[0]
        direction = split_line[2].split()[2]
        beg = int(split_line[2].split()[0])
        end = int(split_line[2].split()[1])
        print(name, direction, beg, end)
        SAL.plot_individual_segment(beg, end, [],\
                                [('Control', fwd_dicty_ctrl, rev_dicty_ctrl), ('Treatment', fwd_dicty_treatment, rev_dicty_treatment)],\
                                defined_ylimits=defined_ylimits, feature_name=name+' '+direction, save_file=save_file)
    except IndexError:
        print(line)
    

# Analysis of much higher quality sRNAs

In [ ]:
save_file = False
defined_ylimits=False
srna_df = pd.read_csv('../Data/short_candidate_srna_neisseria.csv', delimiter=',',\
                        names=['Name', 'Beginning', 'End', 'Strand'] )
srna_df
for index,row in srna_df.iterrows():
    print(row['Name'])
    if row['Strand']=='+':
        strand = 'plus'
    if row['Strand']=='-':
        strand = 'minus'
    vert_lines= (strand, row['Beginning'], row['End'])
    window_beg = row['Beginning']-500
    window_end = row['End']+500
    feature_list = []
    for feature in feature_dict.values():
        if feature.location.start > window_beg and feature.location.start < window_end:
            feature_list.append(feature)
        elif feature.location.end > window_beg and feature.location.end < window_end:
            feature_list.append(feature)
        elif feature.location.start < window_beg and feature.location.end > window_end:
            feature_list.append(feature)
    SAL.plot_individual_segment(window_beg, window_end, feature_list,\
                            [('Control', fwd_dicty_ctrl, rev_dicty_ctrl), ('Treatment', fwd_dicty_treatment, rev_dicty_treatment)],\
                            defined_ylimits=defined_ylimits, vert_lines=vert_lines, feature_name=row['Name']+' '+row['Strand'], save_file=save_file)

    

In [ ]:
print(genome.seq[5720:6320].reverse_complement())

In [ ]:
plt.figure()
plt.hist([float(i) for i in tss_df['enrichmentFactor']])

In [ ]:
blah = []
for i in tss_df['enrichmentFactor']:
    try:
        tempy = float(i)
        if np.isnan(tempy) == False:
            blah.append(tempy)
    except ValueError:
        pass
print(len(blah))
plt.figure()
plt.hist(blah, 50)

# Kallisto

In [5]:
wiggle_dicty_no_tex = {}
for number in [1, 2, 3, 4, 5, 6]:
    wiggle_dicty_no_tex[number] = {}
    fwd_wiggle = '/Users/adamhockenberry/workspace/kallisto/neisseria_04_22_16/SQ{}normed{}.wig'.format(number, '_f')
    rev_wiggle = '/Users/adamhockenberry/workspace/kallisto/neisseria_04_22_16/SQ{}normed{}.wig'.format(number, '_r')
    fwd_dicty, rev_dicty = SAL.read_single_wiggle(fwd_wiggle, rev_wiggle)
    wiggle_dicty_no_tex[number]['Fwd'] = fwd_dicty
    wiggle_dicty_no_tex[number]['Rev'] = rev_dicty

wiggle_dicty_tex = {}
for number in [7, 8, 9, 10, 11, 12]:
    wiggle_dicty_tex[number] = {}
    fwd_wiggle = '/Users/adamhockenberry/workspace/kallisto/neisseria_04_22_16/SQ{}normed{}.wig'.format(number, '_f')
    rev_wiggle = '/Users/adamhockenberry/workspace/kallisto/neisseria_04_22_16/SQ{}normed{}.wig'.format(number, '_r')
    fwd_dicty, rev_dicty = SAL.read_single_wiggle(fwd_wiggle, rev_wiggle)
    wiggle_dicty_tex[number]['Fwd'] = fwd_dicty
    wiggle_dicty_tex[number]['Rev'] = rev_dicty


In [6]:
ctrl_dicty_no_tex = {}
samples = [1, 3, 5]
for strand in ['Fwd', 'Rev']:
    ctrl_dicty_no_tex[strand] = {}
    for i in range(len(genome.seq)):
        tempy = []
        for sample in samples:
            try:
                tempy.append(wiggle_dicty_no_tex[sample][strand][i])
            except KeyError:
                tempy.append(0)
        ctrl_dicty_no_tex[strand][i] = np.mean(tempy)
        
treatment_dicty_no_tex = {}
samples = [2, 4, 6]
for strand in ['Fwd', 'Rev']:
    treatment_dicty_no_tex[strand] = {}
    for i in range(len(genome.seq)):
        tempy = []
        for sample in samples:
            try:
                tempy.append(wiggle_dicty_no_tex[sample][strand][i])
            except KeyError:
                tempy.append(0)
        treatment_dicty_no_tex[strand][i] = np.mean(tempy)

In [7]:
coverage_dict = {}
for key, feature in genome_features_dict.items():
    sequencing_coverage = []
    if feature.type == 'gene':
        if feature.strand == 1:
            for i in range(int(feature.location.start), int(feature.location.end)):
                sequencing_coverage.append(ctrl_dicty_no_tex['Fwd'][i])
            coverage_dict[feature.qualifiers['locus_tag'][0]] = sequencing_coverage
        elif feature.strand == -1:
            for i in range(int(feature.location.start), int(feature.location.end)):
                    sequencing_coverage.append(ctrl_dicty_no_tex['Rev'][i])
            coverage_dict[feature.qualifiers['locus_tag'][0]] = sequencing_coverage[::-1]

In [14]:
import json
with open('../temp_coverage_dict.json', 'w') as outfile:
    json.dump(coverage_dict, outfile)

In [ ]:
ctrl_dicty_tex = {}
samples = [7, 9, 11]
for strand in ['Fwd', 'Rev']:
    ctrl_dicty_tex[strand] = {}
    for i in range(len(genome.seq)):
        tempy = []
        for sample in samples:
            try:
                tempy.append(wiggle_dicty_tex[sample][strand][i])
            except KeyError:
                tempy.append(0)
        ctrl_dicty_tex[strand][i] = np.mean(tempy)
        
treatment_dicty_tex = {}
samples = [10, 12]
for strand in ['Fwd', 'Rev']:
    treatment_dicty_tex[strand] = {}
    for i in range(len(genome.seq)):
        tempy = []
        for sample in samples:
            try:
                tempy.append(wiggle_dicty_tex[sample][strand][i])
            except KeyError:
                tempy.append(0)
        treatment_dicty_tex[strand][i] = np.mean(tempy)

In [ ]:
window_beg, window_end = (531770-100, 532720+100)
# window_beg, window_end = (1014597-100, 1015079+100)
# defined_ylimits = (5000, 5000)
# defined_ylimits = (2000, 2000)
# defined_ylimits = (200, 200)
defined_ylimits = False
logy = False

feature_list = SAL.get_features_in_window(genome_features_dict, window_beg, window_end)

feature_name=''
save_file=True
# feature_name = 'NGO1055 (-TEX)'
feature_name = 'NGO0554'

SAL.plot_individual_segment(window_beg, window_end, feature_list,\
                                [('+ TEX / - H2O2', ctrl_dicty_tex['Fwd'], ctrl_dicty_tex['Rev'], 'r'),\
                                ('- TEX / - H2O2', ctrl_dicty_no_tex['Fwd'], ctrl_dicty_no_tex['Rev'], 'b'),\
                                ('+ TEX / + H2O2', treatment_dicty_tex['Fwd'], treatment_dicty_tex['Rev'], 'm'),\
                                ('- TEX / + H2O2', treatment_dicty_no_tex['Fwd'], treatment_dicty_no_tex['Rev'], 'c')],\
                                defined_ylimits=defined_ylimits, logy=logy, vert_lines='', feature_name=feature_name,\
                            save_file=save_file)

# # feature_name = 'NGO1055 (+TEX)'
# feature_name = 'NGO0554 (+H2O2)'

# SAL.plot_individual_segment(window_beg, window_end, feature_list,\
#                                 [('+ TEX', treatment_dicty_tex['Fwd'], treatment_dicty_tex['Rev'], 'm'),\
#                                 ('- TEX', treatment_dicty_no_tex['Fwd'], treatment_dicty_no_tex['Rev'], 'c')],\
#                                 defined_ylimits=defined_ylimits, logy=logy, vert_lines='', feature_name=feature_name,\
#                             save_file=save_file)

# for i in [1, 3, 5, 4, 6]:
#     fwd_dicty = wiggle_dicty_no_tex[i]['Fwd']
#     rev_dicty = wiggle_dicty_no_tex[i]['Rev']
#     SAL.plot_individual_segment(window_beg, window_end, feature_list,\
#                                     [('Control', fwd_dicty, rev_dicty)],\
#                                     defined_ylimits=defined_ylimits, logy=logy, vert_lines='', feature_name=feature_name,\
#                                 save_file=save_file)

# for i in [7, 9, 11, 10, 12]:
#     fwd_dicty = wiggle_dicty_tex[i]['Fwd']
#     rev_dicty = wiggle_dicty_tex[i]['Rev']
#     SAL.plot_individual_segment(window_beg, window_end, feature_list,\
#                                     [('Control', fwd_dicty, rev_dicty)],\
#                                     defined_ylimits=defined_ylimits, logy=logy, vert_lines='', feature_name=feature_name,\
#                                 save_file=save_file)

# ###To show the feature names
feature_list.sort(key=lambda x: x.location.start)
print('Features shown on plot:')
print('Plus strand:')
for feature in feature_list:
    if feature.strand == 1:
        print(feature.qualifiers['locus_tag'][0], feature.location.start, feature.location.end)
print('Minus strand:')
for feature in feature_list:
    if feature.strand == -1:
        print(feature.qualifiers['locus_tag'][0], feature.location.start, feature.location.end)

In [ ]:
ctrl_dicty_no_tex['Fwd'].keys()

In [ ]:
genome_file = '/Users/adamhockenberry/Projects/Neisseria/Data/Genomes/fa1090.gb'
genome_old = list(SeqIO.parse(genome_file, 'genbank'))[0]
organism = 'Neisseria'
genome_features_dict_old = SAL.get_genome_features(genome_old)

genome_file = '/Users/adamhockenberry/Projects/Neisseria/Code/temp_new_fa1090.gb'
genome_new = list(SeqIO.parse(genome_file, 'genbank'))[0]
organism = 'Neisseria'
genome_features_dict_new = SAL.get_genome_features(genome_new)

In [ ]:
data = {}
found = 0
not_found = 0
for new_feature in genome_features_dict_new:
    temp_start = []
    temp_stop = []
    for old_feature in genome_features_dict_old:
        if genome_features_dict_new[new_feature].location.start == genome_features_dict_old[old_feature].location.start:
            temp_start.append(old_feature)
        if genome_features_dict_new[new_feature].location.end == genome_features_dict_old[old_feature].location.end:
            temp_stop.append(old_feature)
    if len(temp_start) == 1 and len(temp_stop) == 1 and temp_start == temp_stop: 
        data[new_feature] = (temp_start[0], 'Perfect')        
        
    elif len(temp_start) == 1 and len(temp_stop) == 0:
        data[new_feature] = (temp_start[0], 'Same start different stop')        

    elif len(temp_start) == 0 and len(temp_stop) == 1:
        data[new_feature] = (temp_stop[0], 'Same stop different start')
        
    elif len(temp_start) == 0 and len(temp_stop) == 0:
        data[new_feature] = ('NA', 'NA')
    else:
        print(new_feature, temp_start, temp_stop)
            

In [ ]:
df_new = pd.DataFrame()
df_new = df_new.from_dict(data,orient='index')
df_new = df_new.sort_index()
df_new.columns = ['Old ID', 'Note']

df_new.to_excel('new_to_old_annotation.xlsx', sheet_name='Sheet1')

In [ ]:
not_found

In [ ]:
temp

In [ ]:
print(len(genome_features_dict_new))

In [ ]:
genome_features_dict_new['XNG0995'].location.end